In [1]:
! pip install mysql-connector-python
import mysql.connector
import pandas as pd
import json

distributed 1.21.8 requires msgpack, which is not installed.
astroid 2.3.3 has requirement six~=1.12, but you'll have six 1.11.0 which is incompatible.
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
con = open('ref_con/db-prima', 'r').read().splitlines()
how = input('how to connect(local/public):')

mydb = mysql.connector.connect(
  host=con[0] if how == 'public' else con[1],
  user=con[2],
  password=con[3],
  database=con[4] 
)

mycursor = mydb.cursor(dictionary=True)

how to connect(local/public):local


In [12]:
def exec_q(qT):
  mycursor.execute(qT)
  myresult = mycursor.fetchall()
  jD = []
  for x in myresult:
    jD.append(x['json_result'])
  JReturn = '['+','.join(jD)+']'
  return JReturn

def get_erm_result(code,v='',limit=''):
    verQ = ' TRUE ' if v=='' else ' versi = "'+v+'" '
    limQ = '' if limit=='' else ' LIMIT '+limit 
    return exec_q("select json_result from primaprod.mr_result inner join primaprod.mr_form on \
id_form=mr_form.id where kode like '"+code+"' AND "+verQ+limQ)

In [21]:
DFresult = pd.read_json(get_erm_result('RM-B01','','100'))

In [22]:
DFresult

,nama,jenis_kelamin,tekanan_darah,nadi,nafas_,suhu_,saturasi_oksigen,gcs,riwayat_alergi_obat,riwayat_alergi_makanan,...,tanggal_dan_jam_dari_system_,nama_pengantar,hubungan_dengan_pasien_,_spgdt_id,keluhan_utama,systolic,diastolic,kategori_ats,ats_21,_nama_user
0,AGUS W,L,120/90,34,34,34,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOLMES,L,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NAKITA,P,,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SAURON,L,NIHIL,NIHIL,DI MATA,MINUS,,,HOBBIT DRINK,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TN. TEST 2,L,NaN,88,30,36,85,15,tidak ada,tidak ada,...,,,,10,"pusing, musal",120,70,"SKALA 1, SEGERA, Resusitasi",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ny Endang Wahyu Kinasih,P,NaN,90,26,"36,5",97% NRM 15 lpm,15,DIsangkal,Disangkal,...,,DR ASITA,TENAGA KESEHATAN,807,"SESAK NAFAS, G1P0A0 HAMIL 28-29 MINGGU\nKOMORB...",117,80,"SKALA 2, 10 menit, Emergency / Gawat darurat",NaN,dokter igd
96,Ny SUDARMI,P,NaN,78,20,"37,1",92% TANPA OKSIGEN,15,TIDAK ADA,TIDAK ADA,...,,TN TUTAN,ANAK,,"BATUK ADA, LEMAS, RIWAYAT KONTAK PASIEN COVID-...",135,63,"SKALA 3, 30 menit, Urgent / Darurat",NaN,dokter igd
97,fina ratih,P,NaN,100,20,36.2,95-96%,15,tidak ada,tidak ada,...,,pasien sendiri,,,"pasien walk in. demam 3 hari, disertai nyeri o...",164,112,"SKALA 4, 60 menit, Semi darurat",NaN,dokter igd
98,muhamad ridwan,L,NaN,98,20,36,95-96,15,tidak ada,tidak ada,...,,nurlaeli,istri,,"pasien rujukan dr PKM kp Melayu, dx covid 19, ...",131,75,"SKALA 3, 30 menit, Urgent / Darurat",NaN,dokter igd
